In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('pbg20-21data.txt')
winter2 = pd.read_csv ('pbg21-22data.txt')
winter3 = pd.read_csv ('pbg22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
1,PBG,2021-12-01 01:53:00,T,-SN
2,PBG,2021-12-01 02:41:00,T,-SN BR
3,PBG,2021-12-01 02:53:00,T,-SN BR
4,PBG,2021-12-01 03:53:00,0.25,-SN BR
5,PBG,2021-12-01 04:02:00,T,-SN BR
...,...,...,...,...
2780,PBG,2022-02-27 21:33:00,T,-SN
2781,PBG,2022-02-27 21:44:00,T,-SN
2782,PBG,2022-02-27 21:48:00,T,-SN
2783,PBG,2022-02-27 21:53:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
pbg = LocationInfo(
    'Plattsburgh' 'New York', 
    latitude= 44.6821,
    longitude= -73.4466, 
    timezone='America/New_York'
)

pbg

LocationInfo(name='PlattsburghNew York', region='England', timezone='America/New_York', latitude=44.6821, longitude=-73.4466)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(pbg.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(pbg.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(pbg.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:11:34  2020-12-01 16:14:17  2020-12-01
1   2020-12-02 07:12:41  2020-12-02 16:13:58  2020-12-02
2   2020-12-03 07:13:46  2020-12-03 16:13:41  2020-12-03
3   2020-12-04 07:14:50  2020-12-04 16:13:27  2020-12-04
4   2020-12-05 07:15:52  2020-12-05 16:13:15  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:39:35  2021-02-24 17:34:55  2021-02-24
86  2021-02-25 06:37:55  2021-02-25 17:36:16  2021-02-25
87  2021-02-26 06:36:14  2021-02-26 17:37:38  2021-02-26
88  2021-02-27 06:34:31  2021-02-27 17:38:58  2021-02-27
89  2021-02-28 06:32:48  2021-02-28 17:40:19  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:11:18  2021-12-01 16:14:22  2021-12-01
1   2021-12-02 07:12:25  2021-12-02 16:14:02  2021-12-02
2   2021-12-03 07:13:30  2021-12-03 16:13:45  2021-12-03
3   2021-12-04 07:14:35  2021-12-04 16:13:30  2021-12-04
4   2021-12-05 07:15:37  2021-12-05 16:13:18  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:40:00  2022-02-24 17:34:35  2022-02-24
86  2022-02-25 06:38:19  2022-02-25 17:35:56  2022-02-25
87  2022-02-26 06:36:38  2022-02-26 17:37:18  2022-02-26
88  2022-02-27 06:34:56  2022-02-27 17:38:39  2022-02-27
89  2022-02-28 06:33:13  2022-02-28 17:40:00  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:11:01  2022-12-01 16:14:27  2022-12-01
1   2022-12-02 07:12:09  2022-12-02 16:14:06  2022-12-02
2   2022-12-03 07:13:15  2022-12-03 16:13:48  2022-12-03
3   2022-12-04 07:14:19  2022-12-04 16:13:33  2022-12-04
4   2022-12-05 07:15:22  2022-12-05 16:13:20  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:40:24  2023-02-24 17:34:15  2023-02-24
86  2023-02-25 06:38:44  2023-02-25 17:35:37  2023-02-25
87  2023-02-26 06:37:03  2023-02-26 17:36:58  2023-02-26
88  2023-02-27 06:35:21  2023-02-27 17:38:19  2023-02-27
89  2023-02-28 06:33:38  2023-02-28 17:39:40  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PBG,2020-12-02 19:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:12:41,2020-12-02 16:13:58
1,PBG,2020-12-08 09:53:00,0.0001,-SN,2020-12-08,2020-12-08 07:18:50,2020-12-08 16:12:56
2,PBG,2020-12-08 10:53:00,0.0001,-SN,2020-12-08,2020-12-08 07:18:50,2020-12-08 16:12:56
3,PBG,2020-12-09 11:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
4,PBG,2020-12-09 12:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
...,...,...,...,...,...,...,...
158,PBG,2021-02-20 01:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:46:08,2021-02-20 17:29:26
159,PBG,2021-02-23 00:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:41:15,2021-02-23 17:33:33
160,PBG,2021-02-25 06:53:00,0.0001,-SN,2021-02-25,2021-02-25 06:37:55,2021-02-25 17:36:16
161,PBG,2021-02-25 07:53:00,0.2500,-SN BR,2021-02-25,2021-02-25 06:37:55,2021-02-25 17:36:16


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PBG,2021-12-01 01:53:00,0.0001,-SN,2021-12-01,2021-12-01 07:11:18,2021-12-01 16:14:22
1,PBG,2021-12-01 02:53:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:11:18,2021-12-01 16:14:22
2,PBG,2021-12-01 03:53:00,0.2500,-SN BR,2021-12-01,2021-12-01 07:11:18,2021-12-01 16:14:22
3,PBG,2021-12-01 04:53:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:11:18,2021-12-01 16:14:22
4,PBG,2021-12-01 12:53:00,0.2500,-SN,2021-12-01,2021-12-01 07:11:18,2021-12-01 16:14:22
...,...,...,...,...,...,...,...
178,PBG,2022-02-27 11:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:34:56,2022-02-27 17:38:39
179,PBG,2022-02-27 14:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:34:56,2022-02-27 17:38:39
180,PBG,2022-02-27 19:53:00,0.0001,+SN,2022-02-27,2022-02-27 06:34:56,2022-02-27 17:38:39
181,PBG,2022-02-27 20:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:34:56,2022-02-27 17:38:39


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PBG,2022-12-01 13:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:11:01,2022-12-01 16:14:27
1,PBG,2022-12-01 17:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:11:01,2022-12-01 16:14:27
2,PBG,2022-12-10 15:53:00,0.0001,-SN,2022-12-10,2022-12-10 07:20:14,2022-12-10 16:12:55
3,PBG,2022-12-11 17:53:00,0.0001,-SN,2022-12-11,2022-12-11 07:21:07,2022-12-11 16:12:58
4,PBG,2022-12-11 18:53:00,0.0001,-SN,2022-12-11,2022-12-11 07:21:07,2022-12-11 16:12:58
...,...,...,...,...,...,...,...
170,PBG,2023-02-26 17:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:37:03,2023-02-26 17:36:58
171,PBG,2023-02-26 18:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:37:03,2023-02-26 17:36:58
172,PBG,2023-02-27 02:53:00,0.0001,-SN BR,2023-02-27,2023-02-27 06:35:21,2023-02-27 17:38:19
173,PBG,2023-02-27 03:53:00,0.0001,-SN BR,2023-02-27,2023-02-27 06:35:21,2023-02-27 17:38:19


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
1,PBG,2020-12-08 09:53:00,0.0001,-SN,2020-12-08,2020-12-08 07:18:50,2020-12-08 16:12:56
2,PBG,2020-12-08 10:53:00,0.0001,-SN,2020-12-08,2020-12-08 07:18:50,2020-12-08 16:12:56
3,PBG,2020-12-09 11:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
4,PBG,2020-12-09 12:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
5,PBG,2020-12-09 13:53:00,0.2500,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
...,...,...,...,...,...,...,...
150,PBG,2021-02-16 09:53:00,2.5400,-SN BR,2021-02-16,2021-02-16 06:52:24,2021-02-16 17:23:52
151,PBG,2021-02-16 10:53:00,0.7600,-SN BR,2021-02-16,2021-02-16 06:52:24,2021-02-16 17:23:52
160,PBG,2021-02-25 06:53:00,0.0001,-SN,2021-02-25,2021-02-25 06:37:55,2021-02-25 17:36:16
161,PBG,2021-02-25 07:53:00,0.2500,-SN BR,2021-02-25,2021-02-25 06:37:55,2021-02-25 17:36:16


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
1,PBG,2020-12-08 09:53:00,0.0001,-SN,2020-12-08,2020-12-08 07:18:50,2020-12-08 16:12:56,2020-12-08 08:18:50,2020-12-08 15:12:56
2,PBG,2020-12-08 10:53:00,0.0001,-SN,2020-12-08,2020-12-08 07:18:50,2020-12-08 16:12:56,2020-12-08 08:18:50,2020-12-08 15:12:56
3,PBG,2020-12-09 11:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55,2020-12-09 08:19:46,2020-12-09 15:12:55
4,PBG,2020-12-09 12:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55,2020-12-09 08:19:46,2020-12-09 15:12:55
5,PBG,2020-12-09 13:53:00,0.2500,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55,2020-12-09 08:19:46,2020-12-09 15:12:55
...,...,...,...,...,...,...,...,...,...
149,PBG,2021-02-16 08:53:00,0.5100,SN FZFG,2021-02-16,2021-02-16 06:52:24,2021-02-16 17:23:52,2021-02-16 07:52:24,2021-02-16 16:23:52
150,PBG,2021-02-16 09:53:00,2.5400,-SN BR,2021-02-16,2021-02-16 06:52:24,2021-02-16 17:23:52,2021-02-16 07:52:24,2021-02-16 16:23:52
151,PBG,2021-02-16 10:53:00,0.7600,-SN BR,2021-02-16,2021-02-16 06:52:24,2021-02-16 17:23:52,2021-02-16 07:52:24,2021-02-16 16:23:52
161,PBG,2021-02-25 07:53:00,0.2500,-SN BR,2021-02-25,2021-02-25 06:37:55,2021-02-25 17:36:16,2021-02-25 07:37:55,2021-02-25 16:36:16


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4784180327868853
2021-2022: 0.22613035714285715
2022-2023: 0.3556066666666667


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PBG,2020-12-02 19:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:12:41,2020-12-02 16:13:58
8,PBG,2020-12-09 16:53:00,0.2500,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
9,PBG,2020-12-09 18:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
10,PBG,2020-12-09 19:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
11,PBG,2020-12-09 20:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:19:46,2020-12-09 16:12:55
...,...,...,...,...,...,...,...
155,PBG,2021-02-19 22:53:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:47:44,2021-02-19 17:28:03
156,PBG,2021-02-19 23:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:47:44,2021-02-19 17:28:03
157,PBG,2021-02-20 00:53:00,0.2500,-SN BR,2021-02-20,2021-02-20 06:46:08,2021-02-20 17:29:26
158,PBG,2021-02-20 01:53:00,0.0001,-SN BR,2021-02-20,2021-02-20 06:46:08,2021-02-20 17:29:26


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_5252\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.09536037735849054
2021-2022: 0.10418928571428572
2022-2023: 0.314446


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.pbg.csv', index=False)